## Evaluation

Evaluate model on sample evaluation data.

In [2]:
!pip install statsmodels

  Using cached https://files.pythonhosted.org/packages/88/9a/fb4ec96ff528fa6426063b1357b4aad9af4cb7dafc53ac1d5a5762ffeb91/statsmodels-0.12.1-cp38-cp38-macosx_10_13_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ea/0c/5f61f1a3d4385d6bf83b83ea495068857ff8dfb89e74824c6e9eb63286d8/patsy-0.5.1-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df
import warnings
warnings.filterwarnings('ignore')

### Copy Data Locally

In [18]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [19]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x7f9d3ac29400>)

In [20]:
# Load historical data from local file
oxford_data = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [21]:
ID_COLS = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
CASES_COL = ['NewCases']
NPI_COLS = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
NB_LOOKBACK_DAYS = 30
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-11-30")

In [23]:
# Add GeoID column that combines CountryName and RegionName for easier manipulation of data",
oxford_data['GeoID'] = oxford_data['CountryName'] + '__' + oxford_data['RegionName'].astype(str)

# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in NPI_COLS:
    oxford_data.update(oxford_data.groupby(['CountryName', 'RegionName'])[npi_col].ffill().fillna(0))

# Add new cases column
oxford_data['NewCases'] = oxford_data.groupby('GeoID').ConfirmedCases.diff().fillna(0)
# Fill any missing case values by interpolation and setting NaNs to 0
oxford_data.update(oxford_data.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))
# Keep only the id and cases columns
oxford_data = oxford_data[ID_COLS + CASES_COL + NPI_COLS]

In [24]:
oxford_data.head()

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# Make predictions for each country,region pair
geo_pred_dfs = []
for g in hist_cases_df.GeoID.unique():
#for g in ['United States__nan', 'United States__Alabama']:
    if verbose:
        print('\nPredicting for', g)

    # Pull out all relevant data for country c
    hist_cases_gdf = hist_cases_df[hist_cases_df.GeoID == g]
    last_known_date = hist_cases_gdf.Date.max()
    ips_gdf = ips_df[ips_df.GeoID == g]
    past_cases = np.array(hist_cases_gdf[CASES_COL])
    """past_npis = np.array(hist_ips_df[NPI_COLS])
    future_npis = np.array(ips_gdf[NPI_COLS])"""

    # ~TODO~
    # step 1: pre-process hist_cases_gdf into history array for SARIMAX model
    timesteps = hist_cases_gdf['NewCases'].values.tolist()
    timesteps = moving_average(timesteps, 7)
    data = pd.DataFrame(data=timesteps, columns=['new_cases']).squeeze()
    X = data.values

    # step 2: copy from arima_model.py how to fit SARIMAX model with history & perform grid search for hyperparameters 
    best_score, best_cfg, best_pred = float("inf"), None, None
    for p in range(0, 3):
        for d in range(0, 3):
            for q in range(0, 3):
                train, test = X[:-7], X[-7:]
                sarimax_order = (p,d,q)
                # print('trying:', sarimax_order)
                pred = list()
                for t in range(len(test)): # train using rolling window
                    model = SARIMAX(train, order=sarimax_order, initialization='approximate_diffuse')
                    model_fit = model.fit(disp=0)
                    output = model_fit.forecast()
                    yhat = output[0]
                    yhat = max(0, yhat)  # Do not allow predicting negative cases
                    pred.append(yhat)
                    train = np.append(train, yhat)
                error = sqrt(mean_squared_error(test, pred)) # RMSE
                if error < best_score:
                    best_score, best_cfg, best_pred = error, sarimax_order, pred

    # Make prediction for each day
    geo_preds = []
    # Start predicting from start_date, unless there's a gap since last known date
    current_date = min(last_known_date + np.timedelta64(1, 'D'), start_date)
    days_ahead = 0
    dates = []
    while current_date <= end_date:
        model = SARIMAX(X, order=best_cfg, initialization='approximate_diffuse')
        model_fit = model.fit(disp=0)
        output = model_fit.forecast()
        pred = output[0]
        pred = max(0, pred)  # Do not allow predicting negative cases
        X = np.append(X, pred)

        # Prepare data
        """X_cases = past_cases[-NB_LOOKBACK_DAYS:]
        X_npis = past_npis[-NB_LOOKBACK_DAYS:]
        X = np.concatenate([X_cases.flatten(),
                            X_npis.flatten()])

        # Make the prediction (reshape so that sklearn is happy)
        pred = model.predict(X.reshape(1, -1))[0]
        pred = max(0, pred)  # Do not allow predicting negative cases"""
        # Add if it's a requested date
        if current_date >= start_date:
            geo_preds.append(pred)
            dates.append(current_date)
            if verbose:
                print(f"{current_date.strftime('%Y-%m-%d')}: {pred}")
        else:
            if verbose:
                print(f"{current_date.strftime('%Y-%m-%d')}: {pred} - Skipped (intermediate missing daily cases)")

        # Append the prediction for next day
        # in order to rollout predictions for further days.
        """past_cases = np.append(past_cases, pred)
        past_npis = np.append(past_npis, future_npis[days_ahead:days_ahead + 1], axis=0)"""

        # Move to next day
        current_date = current_date + np.timedelta64(1, 'D')
        days_ahead += 1

    # Create geo_pred_df with pred column
    geo_pred_df = ips_gdf[ID_COLS].copy()
    geo_pred_df['PredictedDailyNewCases'] = geo_preds
    geo_pred_df['CountryName'] = g.split('__')[0]
    geo_pred_df['RegionName'] = g.split('__')[1]
    geo_pred_df['GeoID'] = g
    geo_pred_df['Date'] = dates
    geo_pred_dfs.append(geo_pred_df)

# Combine all predictions into a single dataframe
pred_df = pd.concat(geo_pred_dfs)

# Drop GeoID column to match expected output format
pred_df = pred_df.drop(columns=['GeoID'])

# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [ ]:
%%time
!python predict.py -s 2020-12-01 -e 2020-12-15 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-12-01_2020-12-15.csv

In [39]:
!head predictions/2020-12-01_2020-12-15.csv

CountryName,RegionName,Date,PredictedDailyNewCases
Aruba,nan,2020-12-01,14.952073871634664
Aruba,nan,2020-12-02,14.307590223934163
Aruba,nan,2020-12-03,13.808117290464779
Aruba,nan,2020-12-04,13.240991753033573
Aruba,nan,2020-12-05,12.705430399430888
Aruba,nan,2020-12-06,12.155141510272477
Aruba,nan,2020-12-07,11.611724743334479
Aruba,nan,2020-12-08,11.065101139604401
Aruba,nan,2020-12-09,10.51997407415748


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [12]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [8]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-29
End date: 2021-06-27


In [10]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [13]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-29 to 2021-06-27...

Predicting for Aruba__nan
2020-12-23: 10.621417891094014 - Skipped (intermediate missing daily cases)
2020-12-24: 9.598832496879842 - Skipped (intermediate missing daily cases)
2020-12-25: 9.326071828767807 - Skipped (intermediate missing daily cases)
2020-12-26: 8.972117210673913 - Skipped (intermediate missing daily cases)
2020-12-27: 8.98456870697183 - Skipped (intermediate missing daily cases)
2020-12-28: 8.841584421342905 - Skipped (intermediate missing daily cases)
2020-12-29: 8.915214935927542
2020-12-30: 8.848221793217
2020-12-31: 8.925006697626785
2021-01-01: 8.89142587277828
2021-01-02: 8.958776445289496
2021-01-03: 8.94336679776248
2021-01-04: 9.000572323614547
2021-01-05: 8.996731392620223
2021-01-06: 9.045611998113365
2021-01-07: 9.04970543371749
2021-01-08: 9.092216900690238
2021-01-09: 9.101968515739289
2021-01-10: 9.139761451478158
2021-01-11: 9.15360386045543
2021-01-12: 9.187931247880382
2021-01-13: 9.2047373094

2021-04-20: 0
2021-04-21: 0
2021-04-22: 0
2021-04-23: 0
2021-04-24: 0
2021-04-25: 0
2021-04-26: 0
2021-04-27: 0
2021-04-28: 0
2021-04-29: 0
2021-04-30: 0
2021-05-01: 0
2021-05-02: 0
2021-05-03: 0
2021-05-04: 0
2021-05-05: 0
2021-05-06: 0
2021-05-07: 0
2021-05-08: 0
2021-05-09: 0
2021-05-10: 0
2021-05-11: 0
2021-05-12: 0
2021-05-13: 0
2021-05-14: 0
2021-05-15: 0
2021-05-16: 0
2021-05-17: 0
2021-05-18: 0
2021-05-19: 0
2021-05-20: 0
2021-05-21: 0
2021-05-22: 0
2021-05-23: 0
2021-05-24: 0
2021-05-25: 0
2021-05-26: 0
2021-05-27: 0
2021-05-28: 0
2021-05-29: 0
2021-05-30: 0
2021-05-31: 0
2021-06-01: 0
2021-06-02: 0
2021-06-03: 0
2021-06-04: 0
2021-06-05: 0
2021-06-06: 0
2021-06-07: 0
2021-06-08: 0
2021-06-09: 0
2021-06-10: 0
2021-06-11: 0
2021-06-12: 0
2021-06-13: 0
2021-06-14: 0
2021-06-15: 0
2021-06-16: 0
2021-06-17: 0
2021-06-18: 0
2021-06-19: 0
2021-06-20: 0
2021-06-21: 0
2021-06-22: 0
2021-06-23: 0
2021-06-24: 0
2021-06-25: 0
2021-06-26: 0
2021-06-27: 0

Predicting for Angola__nan
2020-1

2021-02-24: 0
2021-02-25: 0
2021-02-26: 0
2021-02-27: 0
2021-02-28: 0
2021-03-01: 0
2021-03-02: 0
2021-03-03: 0
2021-03-04: 0
2021-03-05: 0
2021-03-06: 0
2021-03-07: 0
2021-03-08: 0
2021-03-09: 0
2021-03-10: 0
2021-03-11: 0
2021-03-12: 0
2021-03-13: 0
2021-03-14: 0
2021-03-15: 0
2021-03-16: 0
2021-03-17: 0
2021-03-18: 0
2021-03-19: 0
2021-03-20: 0
2021-03-21: 0
2021-03-22: 0
2021-03-23: 0
2021-03-24: 0
2021-03-25: 0
2021-03-26: 0
2021-03-27: 0
2021-03-28: 0
2021-03-29: 0
2021-03-30: 0
2021-03-31: 0
2021-04-01: 0
2021-04-02: 0
2021-04-03: 0
2021-04-04: 0
2021-04-05: 0
2021-04-06: 0
2021-04-07: 0
2021-04-08: 0
2021-04-09: 0
2021-04-10: 0
2021-04-11: 0
2021-04-12: 0
2021-04-13: 0
2021-04-14: 0
2021-04-15: 0
2021-04-16: 0
2021-04-17: 0
2021-04-18: 0
2021-04-19: 0
2021-04-20: 0
2021-04-21: 0
2021-04-22: 0
2021-04-23: 0
2021-04-24: 0
2021-04-25: 0
2021-04-26: 0
2021-04-27: 0
2021-04-28: 0
2021-04-29: 0
2021-04-30: 0
2021-05-01: 0
2021-05-02: 0
2021-05-03: 0
2021-05-04: 0
2021-05-05: 0
2021-0

2021-03-22: 1.2888332604590688e-31
2021-03-23: 1.304420042472671e-31
2021-03-24: 1.3200068244862734e-31
2021-03-25: 1.3355936064998758e-31
2021-03-26: 1.351180388513478e-31
2021-03-27: 1.3667671705270804e-31
2021-03-28: 1.3823539525406828e-31
2021-03-29: 1.397940734554285e-31
2021-03-30: 1.4135275165678874e-31
2021-03-31: 1.4291142985814898e-31
2021-04-01: 1.444701080595092e-31
2021-04-02: 1.4602878626086944e-31
2021-04-03: 1.4758746446222968e-31
2021-04-04: 1.491461426635899e-31
2021-04-05: 1.5070482086495014e-31
2021-04-06: 1.5226349906631038e-31
2021-04-07: 1.538221772676706e-31
2021-04-08: 1.5538085546903084e-31
2021-04-09: 1.5693953367039108e-31
2021-04-10: 1.5849821187175131e-31
2021-04-11: 1.6005689007311154e-31
2021-04-12: 1.6161556827447178e-31
2021-04-13: 1.6317424647583201e-31
2021-04-14: 1.6473292467719224e-31
2021-04-15: 1.6629160287855248e-31
2021-04-16: 1.6785028107991271e-31
2021-04-17: 1.6940895928127295e-31
2021-04-18: 1.7096763748263318e-31
2021-04-19: 1.725263156839

2021-05-22: 4336.121272823586
2021-05-23: 4335.095099126021
2021-05-24: 4334.069026369711
2021-05-25: 4333.042984855833
2021-05-26: 4332.016902417635
2021-05-27: 4330.99071609602
2021-05-28: 4329.964380053564
2021-05-29: 4328.937829973076
2021-05-30: 4327.911018774821
2021-05-31: 4326.883906046954
2021-06-01: 4325.856437534412
2021-06-02: 4324.828567506887
2021-06-03: 4323.800260869088
2021-06-04: 4322.77149392294
2021-06-05: 4321.742201785611
2021-06-06: 4320.7123609794135
2021-06-07: 4319.681963400182
2021-06-08: 4318.650961046349
2021-06-09: 4317.619315841618
2021-06-10: 4316.587012520827
2021-06-11: 4315.554046726182
2021-06-12: 4314.520359227559
2021-06-13: 4313.485959228128
2021-06-14: 4312.450811957724
2021-06-15: 4311.414903184994
2021-06-16: 4310.3782265694645
2021-06-17: 4309.340743477933
2021-06-18: 4308.302456293983
2021-06-19: 4307.263361143238
2021-06-20: 4306.223453799273
2021-06-21: 4305.182717051926
2021-06-22: 4304.141162024499
2021-06-23: 4303.098790004776
2021-06-24

...

2021-04-30: 6.854264069462e-13
2021-05-01: 6.91169163744045e-13
2021-05-02: 6.9691192054189e-13
2021-05-03: 7.02654677339735e-13
2021-05-04: 7.0839743413758e-13
2021-05-05: 7.14140190935425e-13
2021-05-06: 7.1988294773327e-13
2021-05-07: 7.25625704531115e-13
2021-05-08: 7.3136846132896e-13
2021-05-09: 7.37111218126805e-13
2021-05-10: 7.4285397492465e-13
2021-05-11: 7.48596731722495e-13
2021-05-12: 7.5433948852034e-13
2021-05-13: 7.60082245318185e-13
2021-05-14: 7.6582500211603e-13
2021-05-15: 7.71567758913875e-13
2021-05-16: 7.7731051571172e-13
2021-05-17: 7.83053272509565e-13
2021-05-18: 7.8879602930741e-13
2021-05-19: 7.94538786105255e-13
2021-05-20: 8.002815429031e-13
2021-05-21: 8.06024299700945e-13
2021-05-22: 8.1176705649879e-13
2021-05-23: 8.17509813296635e-13
2021-05-24: 8.2325257009448e-13
2021-05-25: 8.28995326892325e-13
2021-05-26: 8.3473808369017e-13
2021-05-27: 8.40480840488015e-13
2021-05-28: 8.4622359728586e-13
2021-05-29: 8.51966354083705e-13
2021-05-30: 8.5770911088155

2021-06-19: 3.144899230739585e-11
2021-06-20: 3.163602269445511e-11
2021-06-21: 3.1823053081514376e-11
2021-06-22: 3.201008346857364e-11
2021-06-23: 3.21971138556329e-11
2021-06-24: 3.2384144242692165e-11
2021-06-25: 3.257117462975143e-11
2021-06-26: 3.275820501681069e-11
2021-06-27: 3.2945235403869954e-11

Predicting for United Kingdom__England
2020-12-23: 12402.826421605332 - Skipped (intermediate missing daily cases)
2020-12-24: 12400.514569899178 - Skipped (intermediate missing daily cases)
2020-12-25: 12398.207276308603 - Skipped (intermediate missing daily cases)
2020-12-26: 12395.904826932318 - Skipped (intermediate missing daily cases)
2020-12-27: 12393.60700438525 - Skipped (intermediate missing daily cases)
2020-12-28: 12391.314988216482 - Skipped (intermediate missing daily cases)
2020-12-29: 12389.028324218316
2020-12-30: 12386.746279735147
2020-12-31: 12384.469497480117
2021-01-01: 12382.19774624482
2021-01-02: 12379.931736304856
2021-01-03: 12377.672293817213
2021-01-04: 

2021-02-11: 366.8401989724111
2021-02-12: 366.8401989724111
2021-02-13: 366.8401989724111
2021-02-14: 366.8401989724111
2021-02-15: 366.8401989724111
2021-02-16: 366.8401989724111
2021-02-17: 366.8401989724111
2021-02-18: 366.8401989724111
2021-02-19: 366.8401989724111
2021-02-20: 366.8401989724111
2021-02-21: 366.8401989724111
2021-02-22: 366.8401989724111
2021-02-23: 366.8401989724111
2021-02-24: 366.8401989724111
2021-02-25: 366.8401989724111
2021-02-26: 366.8401989724111
2021-02-27: 366.8401989724111
2021-02-28: 366.8401989724111
2021-03-01: 366.8401989724111
2021-03-02: 366.8401989724111
2021-03-03: 366.84019897241114
2021-03-04: 366.8401989724112
2021-03-05: 366.84019897241126
2021-03-06: 366.84019897241126
2021-03-07: 366.84019897241126
2021-03-08: 366.84019897241126
2021-03-09: 366.8401989724112
2021-03-10: 366.8401989724112
2021-03-11: 366.8401989724112
2021-03-12: 366.8401989724112
2021-03-13: 366.84019897241126
2021-03-14: 366.84019897241126
2021-03-15: 366.84019897241126
20

2021-04-23: 406.82744908869086
2021-04-24: 406.82744908869086
2021-04-25: 406.82744908869086
2021-04-26: 406.82744908869086
2021-04-27: 406.82744908869086
2021-04-28: 406.82744908869086
2021-04-29: 406.82744908869086
2021-04-30: 406.82744908869086
2021-05-01: 406.82744908869086
2021-05-02: 406.82744908869086
2021-05-03: 406.82744908869086
2021-05-04: 406.82744908869086
2021-05-05: 406.82744908869086
2021-05-06: 406.82744908869086
2021-05-07: 406.82744908869086
2021-05-08: 406.82744908869086
2021-05-09: 406.82744908869086
2021-05-10: 406.82744908869086
2021-05-11: 406.82744908869086
2021-05-12: 406.82744908869086
2021-05-13: 406.82744908869086
2021-05-14: 406.82744908869086
2021-05-15: 406.82744908869086
2021-05-16: 406.82744908869086
2021-05-17: 406.82744908869086
2021-05-18: 406.82744908869086
2021-05-19: 406.82744908869086
2021-05-20: 406.82744908869086
2021-05-21: 406.82744908869086
2021-05-22: 406.82744908869086
2021-05-23: 406.82744908869086
2021-05-24: 406.82744908869086
2021-05-

FileNotFoundError: [Errno 2] No such file or directory: 'predictions/val_6_month_future.csv'